# Creating training corpus of french pharmacology texts

We try to obtain the more relevant corpus of text for the training of Fasttext embedding

In [1]:
import numpy as np 
import os 
import pandas as pd
import re

In [4]:
path2extern_data ='../external_data/'

path2EMEAcorpus = path2extern_data + 'EMEA.es-fr.fr'
medicament_list_path = path2extern_data + 'medicaments_france.xls'
medicament_list2_path = './challenge_data/medicList.txt'

## EMEA french corpus

In [5]:
with open(path2EMEAcorpus) as f:
        emea_fr = f.read().splitlines()
# removing main title
emea_fr = emea_fr[3:]

In [6]:
emea_fr[:10]

['ABILIFY',
 'Résumé EPAR à l’ intention du public',
 'Ce document est un résumé du rapport européen public d’ évaluation (EPAR).',
 'Il explique comment le comité des médicaments à usage humain (CHMP) a évalué les études réalisées afin d’ aboutir à ses recommandations relatives aux conditions d’ utilisation du médicament.',
 'Pour plus d’ informations sur votre état ou votre traitement, veuillez consulter la notice (également comprise dans l’ EPAR) ou contacter votre médecin ou votre pharmacien.',
 'Si vous souhaitez davantage d’ informations sur la base des recommandations du CHMP, veuillez lire la discussion scientifique (également comprise dans l’ EPAR).',
 "Qu’ est -ce qu'Abilify?",
 'Abilify est un médicament qui contient le principe actif aripiprazole.',
 'Il est disponible sous la forme de comprimés de 5 mg, 10 mg, 15 mg et 30 mg, de comprimés orodispersibles (comprimés qui se dissolvent dans la bouche) de 10 mg, 15 mg et 30 mg, sous la forme d’ une solution buvable (1 mg/ ml) 

In [7]:
emea_train = [e for e in emea_fr if len(e.split()) >=3]

In [8]:
emea_train[:20]
print('We discard approximately: ', np.round(1 - len(emea_train)/len(emea_fr), 2), '% of the orginal corpus')

We discard approximately:  0.23 % of the orginal corpus


In [9]:
# lower the title lines of the corpus that are upper case
def lower_titles(s):
    s_list = s.split()
    # detect upercase titles
    up = 0
    for w in s_list:
        if w.lower()!=w:
            up+=1
    #print(up)
    if up == len(s_list):
        return s.lower()
    else:
        return s

In [10]:
emea_train = [lower_titles(s) for s in emea_train]

In [11]:
emea_train[:5]

['Résumé EPAR à l’ intention du public',
 'Ce document est un résumé du rapport européen public d’ évaluation (EPAR).',
 'Il explique comment le comité des médicaments à usage humain (CHMP) a évalué les études réalisées afin d’ aboutir à ses recommandations relatives aux conditions d’ utilisation du médicament.',
 'Pour plus d’ informations sur votre état ou votre traitement, veuillez consulter la notice (également comprise dans l’ EPAR) ou contacter votre médecin ou votre pharmacien.',
 'Si vous souhaitez davantage d’ informations sur la base des recommandations du CHMP, veuillez lire la discussion scientifique (également comprise dans l’ EPAR).']

## Add the medicaments list

In [12]:
medic_db = pd.read_excel(medicament_list_path)
medic_db.head()

FileNotFoundError: [Errno 2] No such file or directory: '../external_data/medicaments_france.xls'

In [11]:
# retrieve and concatenate the two interesting columns : 'Dénomination spécialité' , 'Libellé ATC'
medic_train = [(e[2] + ' '+ e[3]).lower() for e in medic_db.values]

In [12]:
# replace ',' by '.' in numerical expressions:
def replace_coma(s):
    reg_num = re.search('\d+,\d+', s)
    while reg_num != None:
        s = re.sub(reg_num.group(0), re.sub(',', '.',reg_num.group(0)), s)
        reg_num = re.search('\d+,\d+', s) 
    return s

In [13]:
medic_train = [replace_coma(s) for s in medic_train]
medic_train[:5]

['ubistesin adrenalinee au 1/100 000, solution injectable pour usage dentaire articaïne en association',
 'ubistesin adrenalinee au 1/200 000, solution injectable pour usage dentaire articaïne en association',
 'forene, liquide pour inhalation par vapeur isoflurane',
 'sevorane, liquide pour inhalation par vapeur sévoflurane',
 'chirocaïne 2.5 mg/ml, solution injectable ou solution à diluer pour perfusion lévobupivacaïne']

## Other corpus to add? Add them here! 

In [14]:
medic2_train = []
with open(medicament_list2_path) as f:
    for line in f:
        medic2_train.append(line.lower().rstrip())
medic2_train[:10]

['a 313 200  pour cent',
 'a 313 50 000 u.i',
 'abacavir',
 'abacavir/lamivudine',
 'abacavir/lamivudine pharma',
 'abacavir/lamivudine pharos',
 'abamipharm',
 'abboticine',
 'abelcet',
 'abstral']

## Write final training corpus

In [15]:
path2output = path2extern_data + 'training_fr.txt'
with open(path2output, 'w') as out:
    for s in emea_train:
        out.write(s + '\n')
    for s in medic_train:
        out.write(s + '\n')
    for s in medic2_train:
        out.write(s + '\n')
    for s in medic2_train:
        out.write(s + '\n')
    for s in medic2_train:
        out.write(s + '\n')
    for s in medic2_train:
        out.write(s + '\n')
    for s in medic2_train:
        out.write(s + '\n')
    
    # we add 5 time the medicaments because it is needed to be retained in fasttext dictionnray